In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.conv_learner import *
import fastai.custom_extensions

In [3]:
# architecture variables
PATH = "data/tree_identifier/"

arch=resnet34
sz=128
bs=128
filename='resnet34'

In [5]:
!rm -r {PATH}tmp

In [6]:
# model variables
def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz=sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_paths(PATH, bs=bs, tfms=tfms)
    return data #if sz>300 else data.resize(340, 'tmp')

In [7]:
# dataset
data = get_data(sz, bs)

In [ ]:
# show random images from training set (for giggles)
# fname = random.choice(data.trn_ds.fnames)
# img = PIL.Image.open(PATH + fname)
# display(img); display(img.size)

In [ ]:
# CLEAN BAD IMAGE DATA

def remove_file(fname):
    print(fname)
    !rm {fname}
   
def clean_data(ds):
    for img in ds:
        try: 
            PIL.Image.open(PATH+img)
        except IOError as error:
            str = error.args[0][27:]
            fname = re.sub(r'\\\\(.)', r'\1', str)
            # needs to be unescaped
            remove_file(fname)
            
clean_data(data.val_ds.fnames)
clean_data(data.trn_ds.fnames)

In [ ]:
def plots(ims, figsize=None, rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('On')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])
        
def load_img_id(ds, path): return np.array(PIL.Image.open(PATH+path))

def rand_imgs(ds, num):
    paths = np.random.choice(ds, num, replace=False)
    imgs = [load_img_id(ds,p) for p in paths]
    titles = np.array([t[6:-14] for t in paths])
    rows = -(-num//4)
#     upside down floor division -> ceiling division  https://stackoverflow.com/a/17511341
    return plots(imgs, rows=rows, titles=titles, figsize=(16,8))

In [ ]:
rand_imgs(data.trn_ds.fnames, 6)

### Determine sizes

In [ ]:
# create a dictionary comprehension of image sizes in the dataset
size_dist = {k: PIL.Image.open(PATH+k).size for k in ds}

row_sz,col_sz = list(zip(*size_dist.values()))
row_sz = np.array(row_sz); col_sz = np.array(col_sz)

plt.hist(row_sz), plt.hist(col_sz)

In [ ]:
np.mean(row_sz), np.mean(col_sz)

In [ ]:
np.median(row_sz), np.median(col_sz)

In [8]:
# Train last layer
learn = ConvLearner.pretrained(arch, data)

# ps -> dropout parameters
# hyperparameter to control overfitting (train loss < val loss) by dropping random % of nodes in a layer

In [9]:
learn.lr_find()
# learn.sched.plot_lr()
# learn.sched.plot()
plot_loss_change(learn.sched, sma=20, y_lim=(-0.05, 0.01))

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1512387374934/work/torch/lib/THC/generic/THCTensorCopy.c:70

In [ ]:
lr = 1e-1
lrs = np.array([1e-5, 1e-4, lr])

In [ ]:
# SGDR (one restart per epoch)
learn.fit(lr, 5, cycle_len=1)

# start doing our usual minibatch gradient descent with a given learning rate (lr)
# while gradually decreasing it (fast.ai uses “cosine annealing”) until the end of the cycle

In [ ]:
learn.save('224v4_last')

In [ ]:
learn.load('224v4_last')

In [ ]:
learn.set_data(get_data(400, 50))
learn.lr_find()
plot_loss_change(learn.sched, sma=20, y_lim=(-0.01, 0.01))

In [ ]:
learn.freeze()
learn.fit(lr, 3, cycle_len=1)

In [ ]:
learn.save('400v4_last')

In [ ]:
learn.unfreeze()
learn.bn_freeze(True)
lr = 1e-2
learn.fit(lrs, 3, cycle_len=1)

In [ ]:
learn.load('224v3')

In [ ]:
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
# Test Time Augmentation

log_preds,y = learn.TTA(is_test=True)
probs = np.mean(np.exp(log_preds),0)

In [ ]:
df = pd.DataFrame(probs)
df.columns = data.classes
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [ ]:
SUBM = f'{PATH}subm/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(SUBM+filename, index=False)

In [ ]:
# submit to kaggle via cli
! kg submit {SUBM+filename} -c dog-breed-identification